In [42]:
import openeo

In [43]:
from openeo.rest.auth.config import RefreshTokenStore
RefreshTokenStore().remove()

In [49]:
connection = openeo.connect("openeo.cloud").authenticate_oidc()

Authenticated using refresh token.


In [50]:
s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 4.00, "south": 51.04, "east": 4.10, "north": 51.1},
    temporal_extent=["2022-03-01", "2022-03-31"],
    bands=["B02", "B03", "B04"]
)

In [51]:
# Create a UDF object from inline source code.

udf = openeo.UDF("""

import xarray


def apply_datacube(cube: xarray.DataArray, context: dict) -> xarray.DataArray:

    cube.values = 0.0001 * cube.values

    return cube

""")

In [52]:
rescaled = s2_cube.apply(process=udf)

In [53]:
rescaled.download("apply-udf-scaling.nc")

OpenEoApiError: [401] TokenExchangeFailure: Authentication failed. Please login on our marketplace web https://portal.terrascope.be first, and then try authenticate again. (ref: r-241128b85ff541babe599708f7851079)